In [1]:
!pip install datasets

In [2]:
# from datasets import load_dataset

# dataset = load_dataset("kritsadaK/EDGAR-CORPUS-Financial-Summarization")

In [3]:
# %%
!pip install transformers datasets evaluate sentence_transformers rouge_score bert_score scikit-learn streamlit

# %%
import torch
import os
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on: {device}")

# Load tokenizer and model
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

# Load dataset
dataset = load_dataset("kritsadaK/EDGAR-CORPUS-Financial-Summarization")

# Convert dataset to Pandas DataFrame
df = dataset["train"].to_pandas()

# Rename columns (Ensure they match the dataset's column names)
df = df.rename(columns={"article": "input", "summary": "summary"})

# Split dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Wrap datasets inside a DatasetDict
dataset_dict = DatasetDict({"train": train_dataset, "test": test_dataset})

# Preprocessing function
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input"], max_length=1024, truncation=True, padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=256, truncation=True, padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize datasets
tokenized_train_dataset = dataset_dict["train"].map(preprocess_function, batched=True)
tokenized_test_dataset = dataset_dict["test"].map(preprocess_function, batched=True)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./bart_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./bart_logs",
    save_total_limit=2,
    report_to="none",
    fp16=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

# Train model
trainer.train()

# Save model & tokenizer
model.save_pretrained("./fine_tuned_bart_model")
tokenizer.save_pretrained("./fine_tuned_bart_model")

print("Training complete and model saved.")


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 82.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.2 MB/s eta 0:00:0000:01
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=0077d2ed7354b026cc9b41fea6b5eba94d30d038c7bc5fe84240d56cb84006cb
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Training on: cuda


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

...

Map:   0%|          | 0/1061 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.244900,1.197466
2,1.048100,1.102999
3,0.913900,1.074477


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Training complete and model saved.


In [4]:
# %%
!pip install transformers datasets evaluate sentence_transformers rouge_score bert_score scikit-learn nltk

# %%
import torch
import os
import numpy as np
import evaluate
from datasets import load_dataset, Dataset, DatasetDict
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import corpus_bleu

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Evaluating on: {device}")

# Load tokenizer and model
model_path = "./fine_tuned_bart_model"
tokenizer = BartTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path).to(device)

# Load dataset
dataset = load_dataset("kritsadaK/EDGAR-CORPUS-Financial-Summarization")

# Convert dataset to Pandas DataFrame
df = dataset["train"].to_pandas()

# Rename columns
df = df.rename(columns={"article": "input", "summary": "summary"})

# Split dataset into test set (use only test set for evaluation)
_, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_df)

# Preprocessing function
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input"], max_length=1024, truncation=True, padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=256, truncation=True, padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize test dataset
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

# Load evaluation metrics
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# Function to generate predictions
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    summary_ids = model.generate(inputs.input_ids, max_length=256, min_length=50, num_beams=4, length_penalty=2.0)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Evaluate model
references = []
predictions = []

for example in test_df.sample(100).itertuples():  # Sample 100 examples for faster evaluation
    input_text = example.input
    reference_summary = example.summary
    generated_summary = generate_summary(input_text)
    
    references.append([reference_summary])  # BLEU requires list of lists
    predictions.append(generated_summary)

# Compute ROUGE Score
rouge_scores = rouge.compute(predictions=predictions, references=[ref[0] for ref in references])
print("ROUGE Scores:", rouge_scores)

# Compute BERTScore
bertscore_results = bertscore.compute(predictions=predictions, references=[ref[0] for ref in references], lang="en")
print("BERTScore (F1 Mean):", np.mean(bertscore_results["f1"]))

# Compute BLEU Score
bleu_score = corpus_bleu(references, predictions)
print("BLEU Score:", bleu_score)


Evaluating on: cuda


/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


Map:   0%|          | 0/1061 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1527: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore (F1 Mean): 0.5319436860084533
BLEU Score: 0.26002396504062025


In [5]:
!zip -r fine_tuned_bart_model.zip ./fine_tuned_bart_model

  adding: fine_tuned_bart_model/ (stored 0%)
  adding: fine_tuned_bart_model/merges.txt (deflated 53%)
  adding: fine_tuned_bart_model/vocab.json (deflated 68%)
  adding: fine_tuned_bart_model/model.safetensors (deflated 7%)
  adding: fine_tuned_bart_model/tokenizer_config.json (deflated 75%)
  adding: fine_tuned_bart_model/special_tokens_map.json (deflated 85%)
  adding: fine_tuned_bart_model/generation_config.json (deflated 47%)
  adding: fine_tuned_bart_model/config.json (deflated 61%)


In [6]:
from IPython.display import FileLink
FileLink(r'fine_tuned_bart_model.zip')

/kaggle/working/fine_tuned_bart_model.zip